In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import tgt
import os
import numpy as np
from tqdm import tqdm
import librosa
import glob

sys.path.append("../")
from pyfiles.utils import trim_audio_and_save

sys.path.append("../")
from pyfiles.feature_extractor import WavLMExtractor

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"

wavlm = WavLMExtractor()
speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*/*")]
speakers.sort()

In [3]:
save = True
tempfile = "temp3.wav"
trim_threshold_in_db = 30

files = []
for spk in tqdm(speakers):
    files += glob.glob(dataset_dir + f"*/{spk}/*/*.wav")
files.sort()
for path in tqdm(files):
    trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
    embedding = wavlm.get_feature(tempfile).T.detach().cpu().numpy()
    savepath = feat_base_dir + "/".join(path.split("/")[-4:])[:-4] + "_wavlm.npy"
    if save:
        np.save(savepath, embedding)

  0%|          | 389/160267 [01:12<8:18:16,  5.35it/s] 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x155550312d90>>
Traceback (most recent call last):
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



# Normalize

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

In [ ]:
scaler_name = "LibriTTS-R_wavlm"

In [ ]:
scaler = StandardScaler()
files = []
for spk in tqdm(speakers):
    files += glob.glob(feat_base_dir + f"train*/{spk}/*/*[0-9]_wavlm.npy")
files.sort()
for path in tqdm(files):
    mel = np.load(path)
    scaler.partial_fit(mel.T)
            
scaler_filename = f"ckpts/scalers/{scaler_name}.save"
joblib.dump(scaler, scaler_filename) 
# a = joblib.load(scaler_filename)